##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - Prompt Chaining and Iterative Generation
This notebook demonstrates how to use prompt chaining and iterative generation with Gemma through a story writing example.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Prompt_chaining.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma setup

To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run
  the Gemma 2B model.
* Generate and configure a Kaggle username and an API key as Colab secrets.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.


### Configure your credentials

Add your your Kaggle credentials to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create new secrets: `KAGGLE_USERNAME` and `KAGGLE_KEY`
3. Copy/paste your username into `KAGGLE_USERNAME`
3. Copy/paste your key into `KAGGLE_KEY`
4. Toggle the buttons on the left to allow notebook access to the secrets.


In [ ]:
import os
from google.colab import userdata

# Set the backbend before importing Keras
os.environ["KERAS_BACKEND"] = "jax"
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### Install dependencies
Run the cell below to install all the required dependencies.

In [ ]:
!pip install -q -U tensorflow keras keras-nlp

## Prompt chaining

Prompt chaining is a powerful technique for managing complex tasks that are difficult to accomplish in a single step.

It entails breaking a large task into smaller, linked prompts, where each prompt's output feeds into the next. This step-by-step method steers the language model through the process. Key advantages include:



*   Enhanced accuracy: Focused, smaller prompts produce better results from the language model.
*   Easier debugging: Pinpointing and fixing issues within the chain is straightforward, allowing for precise improvements.
* Handling complexity: Dividing intricate problems into manageable steps enables the language model to address more complex tasks.


## Iterative Generation
Iterative generation is the process of creating the desired output step by step. This method is particularly useful for writing stories that exceed the length limitations of a single generation window. The advantages of iterative generation include:

* Extended outputs: It enables the production of longer and more detailed content, going beyond the constraints of a single generation window.
* Enhanced flexibility: Adjustments and refinements can be made at each
iteration, ensuring the story progresses as intended.
* Human oversight: Feedback and guidance can be provided at each step, ensuring the story stays true to the creator's vision.


By using **prompt chaining** and **iterative generation** together, you can create an interesting and well-structured story, adding to it piece by piece, while still having control over how it unfolds.

### Gemma

**About Gemma**

Gemma is a family of lightweight, state-of-the-art open models from Google, built from the same research and technology used to create the Gemini models. They are text-to-text, decoder-only large language models, available in English, with open weights, pre-trained variants, and instruction-tuned variants. Gemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop or your own cloud infrastructure, democratizing access to state of the art AI models and helping foster innovation for everyone.

Here's the [official documentation](https://ai.google.dev/gemma/docs/formatting) regarding prompting instruction-tuned models.

In [ ]:
import keras
import keras_nlp
from IPython.display import display, Markdown, Latex

In [ ]:
# Let's load Gemma using Keras
gemma_model_id = "gemma2_instruct_2b_en"
gemma = keras_nlp.models.GemmaCausalLM.from_preset(gemma_model_id)

## Story writing baseline: persona, premise, outline

As the first step, you'll create a persona, which the LLM should take to perform the task. In this example, you want it to act like a children's author, who wants to write a new funny and educating story.

In [ ]:
persona = """You are a children's author with a penchant for humorous, yet educating stories.
Your ultimate goal is to write a new story to be published in a children magazine."""

The upcoming subtask for the LLM is to develop a premise for the story. To achieve this, you require a prompt. You'll use the persona description to generate the premise prompt. In this example, you want the model to create a story about bunnies.

In [ ]:
premise_prompt = f"""<start_of_turn>user
{persona}

Write a single-sentence premise for a children's story featuring bunnies.<end_of_turn>
<start_of_turn>model\n"""

In [ ]:
premise_response = gemma.generate(premise_prompt, max_length=512)
premise = premise_response[len(premise_prompt) :]
display(Markdown(premise))

When a mischievous bunny accidentally swaps his carrot patch with a giant sunflower field, he must learn the importance of teamwork and communication to get his carrots back. 
<end_of_turn>

You'll use the generated premise to create a prompt for the next step, which will produce the story outline.


In [ ]:
outline_prompt = f"""<start_of_turn>user
{persona}

You have a gripping premise in mind:

{{premise}}

Create an outline for your story's plot consisting of 5 key points.<end_of_turn>
<start_of_turn>model\n"""

In [ ]:
full_outline_prompt = outline_prompt.format(premise=premise)
outline_response = gemma.generate(full_outline_prompt, max_length=512)
outline = outline_response[len(full_outline_prompt) :]
display(Markdown(outline))

Here's a plot outline for a children's story about a mischievous bunny and his carrot patch swap:

**Title:** Benny's Big Sunflower Swap

**Key Points:**

1. **The Mishap:** Benny, a playful bunny with a penchant for mischief, accidentally trips over a root and sends his entire carrot patch tumbling into a giant sunflower field. He's horrified! He can't imagine a world without his delicious carrots.
2. **The Sunflower Surprise:** Benny, bewildered and hungry, explores the sunflower field. He's amazed by the towering sunflowers, but they're not very tasty. He tries to eat the petals, but they're too tough. He even tries to climb the sunflowers, but they're too tall!
3. **The Sunflower Friends:** Benny meets a friendly sunflower named Sunny who explains that the sunflowers are actually quite helpful. They provide shade for the bees, attract butterflies, and even help the birds find food. Benny learns that the sunflower field is a vital part of the ecosystem.
4. **The Teamwork Challenge:** Benny realizes he needs to work with the sunflowers to get his carrots back. He learns that the sunflowers can't move the carrots, but they can help him find a way to get them back. He asks Sunny for help, and together they brainstorm ideas.
5. **The Carrot Rescue:** Benny and Sunny come up with a plan to use the sunflowers' height to reach the carrots. They work together, with Benny using his agility to climb the sunflowers and Sunny guiding him. They successfully retrieve the carrots, and Benny learns the importance of teamwork and communication.


**Humorous Elements:**

* Benny's mischievous nature could lead to funny situations, like him trying to eat the sunflower seeds or trying to climb the sunflowers with his carrot patch stuck to his back.
* Sunny the sunflower could have a quirky personality, perhaps with a love for riddles or a habit of telling tall tales.
* The story could incorporate silly

Once you have the plan, you'd like Gemma to begin writing the story. In the prompt, include all the necessary information you've gathered so far: the persona, premise, and outline.


In [ ]:
starting_prompt = f"""<start_of_turn>user
{persona}

You have a gripping premise in mind:

{{premise}}

Your imagination has crafted a narrative outline:

{{outline}}

First, silently review the outline and the premise. Consider how to start the
story.

Your task is to write a part of the story that covers only the first point of the outline.
You are not expected to finish the whole story now.
Do not write about the next points, only the first one plot point!!!

Try to write 10 sentences.
Remember, DO NOT WRITE A WHOLE STORY RIGHT NOW.<end_of_turn>
<start_of_turn>model\n"""

In [ ]:
full_starting_prompt = starting_prompt.format(premise=premise, outline=outline)
starting_response = gemma.generate(full_starting_prompt, max_length=1000)
draft = starting_response[len(full_starting_prompt) :]
display(Markdown(draft))

Benny the bunny was a whirlwind of fluff and mischief.  He loved nothing more than hopping through his carrot patch, nibbling on the plumpest, juiciest carrots.  But today, Benny's playful antics took a turn for the disastrous.  He tripped over a particularly stubborn root, sending a cascade of carrots tumbling into the air.  Benny watched in horror as his entire patch, his precious treasure, disappeared into a sea of towering sunflowers.  The sunflowers, with their bright yellow faces, seemed to mock him with their endless rows.  Benny's heart sank.  He couldn't imagine a world without his carrots!  He hopped around, his ears drooping, his little nose twitching with despair.  He needed his carrots, and he needed them now!  But how could he possibly get them back? 
<end_of_turn>

If you're pleased with the start of your story, you can continue it by further prompting the model with the text written so far. You can also add guidelines to help the model write appropriately and avoid concluding the story too quickly.

In [ ]:
guidelines = """Writing Guidelines

Remember, your main goal is to write as much as you can. If you get through
the story too fast, that is bad. Expand, never summarize. Don't repeat previous
parts of the story, only expand."""


continuation_prompt = f"""<start_of_turn>user
{persona}

You have a following premise in mind:

{{premise}}

The outline of the story looks like this:

{{outline}}


Here's what you've written so far:

{{story_text}}


=====
First, silently review the premise, the outline and the story you've written so far.

Write the continuation - the next 5 sentences that cover the next outline point. Stick to the outline.

However, once the story is COMPLETELY finished, write IAMDONE.

{guidelines}<end_of_turn>
<start_of_turn>model\n"""

In [ ]:
full_continuation_prompt = continuation_prompt.format(
    premise=premise, outline=outline, story_text=draft
)
continuation_response = gemma.generate(full_continuation_prompt, max_length=1000)
continuation = continuation_response[len(full_continuation_prompt) :]
display(Markdown(continuation))

Benny hopped from sunflower to sunflower, his nose twitching, trying to catch a whiff of his lost carrots.  He sniffed the air, hoping to catch a hint of their sweet scent, but all he could smell was the earthy aroma of the soil and the sweet nectar of the sunflowers.  He felt a pang of sadness, realizing that he couldn't just eat the sunflowers.  They were beautiful, but they weren't carrots!  Benny needed a plan, and he needed it fast. 

 
=====
 
 IAMDONE 
<end_of_turn>

Add the continuation to the initial draft, keep building the story iteratively, until 'IAMDONE' is seen

In [ ]:
draft = draft + "\n\n" + continuation

while "IAMDONE" not in continuation:
    full_continuation_prompt = continuation_prompt.format(
        premise=premise, outline=outline, story_text=draft
    )
    continuation_response = gemma.generate(full_continuation_prompt, max_length=5000)
    continuation = continuation_response[len(full_continuation_prompt) :]
    draft = draft + "\n\n" + continuation

# Remove 'IAMDONE' and print the final story
final = draft.replace("IAMDONE", "").strip()
display(Markdown(final))

Benny the bunny was a whirlwind of fluff and mischief.  He loved nothing more than hopping through his carrot patch, nibbling on the plumpest, juiciest carrots.  But today, Benny's playful antics took a turn for the disastrous.  He tripped over a particularly stubborn root, sending a cascade of carrots tumbling into the air.  Benny watched in horror as his entire patch, his precious treasure, disappeared into a sea of towering sunflowers.  The sunflowers, with their bright yellow faces, seemed to mock him with their endless rows.  Benny's heart sank.  He couldn't imagine a world without his carrots!  He hopped around, his ears drooping, his little nose twitching with despair.  He needed his carrots, and he needed them now!  But how could he possibly get them back? 
<end_of_turn>

Benny hopped from sunflower to sunflower, his nose twitching, trying to catch a whiff of his lost carrots.  He sniffed the air, hoping to catch a hint of their sweet scent, but all he could smell was the earthy aroma of the soil and the sweet nectar of the sunflowers.  He felt a pang of sadness, realizing that he couldn't just eat the sunflowers.  They were beautiful, but they weren't carrots!  Benny needed a plan, and he needed it fast. 

 
=====
 
  
<end_of_turn>